In [28]:
import os
import shutil
import json

from pyparsing import C
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from statistics import mean
import torch
import json
import sys
import time
from NSM.models import TriplanarDecoder
from NSM.mesh import create_mesh
from pymskt.mesh import get_icp_transform, cpd_register, non_rigidly_register
from pymskt import mesh
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy
from pymskt.mesh.meshTools import smooth_scalars_from_second_mesh_onto_base, transfer_mesh_scalars_get_weighted_average_n_closest
from pymskt.mesh import Mesh, BoneMesh, CartilageMesh
from NSM.mesh.interpolate import interpolate_points

In [29]:
path1= '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/change_overtime/BO/change_overtime_aclr/aclr_0.0.vtk'
path2= '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/change_overtime/BO/change_overtime_aclr/aclr_3.0_interp.vtk'

In [31]:
from git import Diff
import pyvista as pv
import numpy as np

def compute_vertex_displacements(mesh1, mesh2, save_path=None):
    """
    Compute x, y, z displacements and magnitude between corresponding points
    in mesh1 and mesh2. Saves results on mesh2.
    """
    # Ensure matching number of points
    assert mesh1.n_points == mesh2.n_points, "Meshes must have the same number of points"

    # Compute displacement vectors
    disp_vectors = mesh2.points - mesh1.points  # shape (N, 3)
    normx = disp_vectors[:, 0]
    normy = disp_vectors[:, 1]
    normz = disp_vectors[:, 2]
    magnitude = np.linalg.norm(disp_vectors, axis=1)

    # Attach as scalars to mesh2
    mesh2.point_data['normx'] = normx
    mesh2.point_data['normy'] = normy
    mesh2.point_data['normz'] = normz
    mesh2.point_data['magnitude'] = magnitude

    return mesh2

# Example usage
mesh1 = pv.read(path1)
mesh2 = pv.read(path2)
out_path = path2

diff_mesh = compute_vertex_displacements(mesh1, mesh2, save_path=out_path)
diff_mesh =  BoneMesh(diff_mesh)
diff_mesh.save_mesh(out_path)

In [43]:
mean_latent_save_path = '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BO_model/change_overtime/mean_latents.npz'
latent= np.load(mean_latent_save_path)
for key in latent.files:
    print(f"{key}: shape = {latent[key].shape}, dtype = {latent[key].dtype}")  

In [44]:
latent

In [ ]:
import pyvista as pv
import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk

import pyvista as pv
import numpy as np
import vtk

def compute_signed_distance(mesh1, mesh2):
    """
    Compute signed distance from each point in mesh2 to the surface of mesh1.
    Signed distance is Positive outside, negative inside, zero on surface.
    Compute normal vector from mesh1 to mesh2.
    """
    
    # Ensure matching number of points
    assert mesh1.n_points == mesh2.n_points, "Meshes must have the same number of points"
    
    # Build implicit distance function from surface of mesh1
    implicit_distance = vtk.vtkImplicitPolyDataDistance()
    surf = mesh1.extract_surface().triangulate().clean()
    implicit_distance.SetInput(surf)  # For older VTK versions

    # Evaluate signed distance at each point in mesh2
    signed_distances = np.array([
        implicit_distance.EvaluateFunction(point) for point in mesh2.points
    ])

    # Store result as point scalar
    mesh2.point_data["signed_distance"] = signed_distances
    
    # Compute displacement vectors
    disp_vectors = mesh2.points - mesh1.points  # shape (N, 3)
    normx = disp_vectors[:, 0]
    normy = disp_vectors[:, 1]
    normz = disp_vectors[:, 2]
    magnitude = np.linalg.norm(disp_vectors, axis=1)

    # Attach as scalars to mesh2
    mesh2.point_data['normx'] = normx
    mesh2.point_data['normy'] = normy
    mesh2.point_data['normz'] = normz
    
    displacement_vectors = np.column_stack([
    mesh2.point_data['normx'],
    mesh2.point_data['normy'],
    mesh2.point_data['normz']
    ])
    mesh2.point_data['displacement'] = displacement_vectors  # single vector field
    
    return mesh2

# Example usage
mesh1 = pv.read(path1)
mesh2 = pv.read(path2)
out_path = path2

diff_mesh = compute_signed_distance(mesh1, mesh2)
diff_mesh =  BoneMesh(diff_mesh)
diff_mesh.save_mesh(out_path)

In [41]:
mean_latent='/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/change_overtime/BO/mean_latent_aclr.npz'

latent_npz = np.load(mean_latent)  # mean_latent = path to .npz file
print(latent_npz.files)  # Lists all keys stored in the npz file

# Optionally, explore contents
for key in latent_npz.files:
    print(f"{key}: shape = {latent_npz[key].shape}, dtype = {latent_npz[key].dtype}")


['Visit-1', 'Visit-2', 'Visit-3', 'Visit-4', 'Visit-5']
Visit-1: shape = (1, 512), dtype = float64
Visit-2: shape = (1, 512), dtype = float64
Visit-3: shape = (1, 512), dtype = float64
Visit-4: shape = (1, 512), dtype = float64
Visit-5: shape = (1, 512), dtype = float64


In [ ]:
import pyvista as pv
import numpy as np

# Load meshes
mesh1 = pv.read(path1)
mesh5 = pv.read(path2)

# Extract triangle indices (drop leading '3' from each face)
faces1 = mesh1.faces.reshape(-1, 4)[:, 1:]
faces5 = mesh5.faces.reshape(-1, 4)[:, 1:]

# Check that all triangles are exactly the same
same_faces = np.array_equal(faces1, faces5)
print("Triangle correspondence check:", same_faces)

# Optional: print mismatches
if not same_faces:
    mismatch = np.where(~np.all(faces1 == faces5, axis=1))[0]
    print(f"Found {len(mismatch)} mismatched triangles. Example:", faces1[mismatch[0]], "vs", faces5[mismatch[0]])


In [ ]:
import pyvista as pv
import numpy as np
import os

def compute_triangle_areas(mesh):
    """
    Compute area of each triangle in a PyVista PolyData mesh.
    Assumes each face is a triangle stored as [3, i, j, k].
    """
    faces = mesh.faces.reshape(-1, 4)[:, 1:]  # Drop the leading '3' per triangle
    points = mesh.points

    v1 = points[faces[:, 1]] - points[faces[:, 0]]
    v2 = points[faces[:, 2]] - points[faces[:, 0]]
    cross_prod = np.cross(v1, v2)
    areas = 0.5 * np.linalg.norm(cross_prod, axis=1)

    return areas, faces

def triangle_to_vertex_scalar(mesh, cell_values):
    """
    Averages cell (triangle) values to point (vertex) values.
    """
    n_points = mesh.points.shape[0]
    point_values = np.zeros(n_points)
    counts = np.zeros(n_points)

    faces = mesh.faces.reshape(-1, 4)[:, 1:]

    for i, tri in enumerate(faces):
        for idx in tri:
            point_values[idx] += cell_values[i]
            counts[idx] += 1

    # Avoid division by zero
    counts[counts == 0] = 1
    return point_values / counts

def triangle_area_change_heatmap(vtk_visit1_path, vtk_visit5_path_restruc, vtk_visit5_path, save_path=None, plot=True):
    # Load meshes
    mesh1 = pv.read(vtk_visit1_path)
    mesh5 = pv.read(vtk_visit5_path_restruc)
    mesh5_org= pv.read(vtk_visit5_path)

    # Compute triangle areas
    area1, _ = compute_triangle_areas(mesh1)
    area5, _ = compute_triangle_areas(mesh5)
    percent_change = 100 * (area5 - area1) / area1
    percent_change = np.clip(percent_change, -50, 50)

    # Convert to per-vertex scalar values
    vertex_values = triangle_to_vertex_scalar(mesh5, percent_change)

    # Assign as point scalars
    mesh5.point_data['area_change (%)'] = vertex_values

    # Save
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        mesh5.save(save_path)
        print(f"Saved mesh with vertex scalars to: {save_path}")

triangle_area_change_heatmap(path1, path2_restructured, path2, save_path)